<a href="https://colab.research.google.com/github/CodeWithSridhar/gdi-agent/blob/main/app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from flask import Flask, request, jsonify, render_template
from werkzeug.utils import secure_filename
import os
from main import OrchestratorAgent
import json

app = Flask(__name__)
app.config['MAX_CONTENT_LENGTH'] = 16 * 1024 * 1024  # 16 MB max file size
app.config['UPLOAD_FOLDER'] = 'uploads'

os.makedirs(app.config['UPLOAD_FOLDER'], exist_ok=True)

orchestrator = OrchestratorAgent()

@app.route('/')
def index():
    """Home page"""
    return jsonify({
        "name": "GDI-Agent API",
        "version": "1.0.0",
        "endpoints": {
            "POST /analyze": "Upload CSV and get analysis",
            "GET /health": "Health check"
        }
    })

@app.route('/health', methods=['GET'])
def health():
    """Health check endpoint"""
    return jsonify({"status": "healthy", "service": "GDI-Agent"})

@app.route('/analyze', methods=['POST'])
def analyze():
    """Analyze uploaded dataset"""
    try:
        # Check if file is in request
        if 'file' not in request.files:
            return jsonify({"error": "No file part"}), 400

        file = request.files['file']
        domain = request.form.get('domain', 'education')

        if file.filename == '':
            return jsonify({"error": "No selected file"}), 400

        if file and file.filename.endswith('.csv'):
            filename = secure_filename(file.filename)
            filepath = os.path.join(app.config['UPLOAD_FOLDER'], filename)
            file.save(filepath)

            # Process with orchestrator
            report = orchestrator.process_dataset(filepath, domain=domain)

            # Cleanup
            os.remove(filepath)

            return jsonify({
                "status": "success",
                "report": report,
                "domain": domain
            }), 200
        else:
            return jsonify({"error": "File must be CSV"}), 400

    except Exception as e:
        return jsonify({"error": str(e)}), 500

if __name__ == '__main__':
    app.run(debug=True, port=8000)
